# Todos

In [ ]:
# Add table of contents

# Libraries and Notebook Options

In [2]:
import pandas as pd
import numpy as np

# Tools
from collections import Counter

# Preprocessing
from sklearn.preprocessing import MinMaxScaler

# Modeling
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from sklearn.svm import LinearSVC, SVC # Remove
from sklearn.datasets import make_classification

# Metrics
from sklearn.metrics import accuracy_score, confusion_matrix

# # Visualizations
# from matplotlib.pyplot import figure
# from sklearn.metrics import roc_auc_score, roc_curve
# import matplotlib.pyplot as plt
# import seaborn as sns
# from mlxtend.plotting import plot_decision_regions
# %matplotlib inline


Using TensorFlow backend.
/Users/miguel/anaconda3/envs/metis/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/miguel/anaconda3/envs/metis/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/miguel/anaconda3/envs/metis/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/miguel/anacon

# Train and Test Splitting

In [3]:
# Load dataframe
df = pd.read_pickle('../data/df_clean_20k.pkl')

df.head()

,case_id,opened,closed,updated,status_notes,responsible_agency,category,request_type,request_details,address,street,supervisor_district,neighborhood,police_district,latitude,longitude,point,source,is_duplicate
619,11879423,2019-12-30 20:54:00,2020-01-03 11:59:19,2020-01-03 11:59:19,Agencies responded to request and no encampmen...,Duplicate Case Hold Queue,Encampments,Encampment Reports,Encampment Cleanup,"3983 19TH ST, SAN FRANCISCO, CA, 94114",19TH ST,8,Dolores Heights,MISSION,37.759285,-122.432162,"(37.759285, -122.432162)",Mobile/Open311,0
620,11877576,2019-12-30 13:31:54,2019-12-31 13:42:03,2019-12-31 13:42:03,Case Resolved - SES Graffiti Crew - Remove Si...,DPW Ops Queue,Illegal Postings,Illegal Postings - Affixed_Improperly,Affixed Improperly,"111 PAGE ST, SAN FRANCISCO, CA, 94102",PAGE ST,5,Lower Haight,NORTHERN,37.773851,-122.423080,"(37.77385123, -122.42307977)",Mobile/Open311,0
621,11877532,2019-12-30 13:26:00,2019-12-30 14:03:00,2019-12-30 14:03:00,Case Resolved,DPW Ops Queue,Street and Sidewalk Cleaning,General Cleaning,Other Loose Garbage,Intersection of OSAGE ALY and 26TH ST,OSAGE ALY,9,Mission,MISSION,37.749008,-122.418709,"(37.74900818, -122.4187088)",Phone,0
622,11877496,2019-12-30 13:22:00,2019-12-30 18:53:45,2019-12-30 18:53:45,Case Resolved - WASTE NOT FOUND ...,Recology_Abandoned,Street and Sidewalk Cleaning,Bulky Items,Refrigerator,Intersection of OSAGE ALY and 25TH ST,OSAGE ALY,9,Mission,MISSION,37.750607,-122.418861,"(37.75060654, -122.41886139)",Phone,0
623,11877234,2019-12-30 12:45:38,2020-01-02 07:17:00,2020-01-02 07:17:00,Case Resolved,DPW Ops Queue,Street and Sidewalk Cleaning,General Cleaning,Other Loose Garbage,Intersection of ELGIN PARK and MCCOPPIN ST,ELGIN PARK,8,Mint Hill,SOUTHERN,37.771481,-122.423325,"(37.77148065, -122.4233246)",Mobile/Open311,0


In [16]:
# Columns to exclude
exclude_cols = [
    'is_duplicate', # Target variable
    'case_id',
    'opened', # Needs Feature Eng
    'closed', # Needs Feature Eng
    'updated',
    'responsible_agency', # Needs NLP
    'status_notes', # Needs NLP
    'request_type', # Needs NLP
    'request_details', # Needs NLP
    'address', # Needs NLP
#     'street', # Convert to 'category' type to get dummies
    'point'
]

# # Scale data using MinMax scaler
# # No need to standardize as all features are categorical (maybe scale lat/long....)
# scaler = MinMaxScaler()

# Predictor variables
x_variables_df = df.drop(columns=exclude_cols, axis=0, inplace=False)

# Get dummies for categorical variables
X = pd.get_dummies(x_variables_df, drop_first=True)

# Target variable
y = df['is_duplicate']

# Split train and test
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2, 
                                                    random_state=2020, 
                                                    stratify=y,  # Stratify to keep same class ratios
                                                    shuffle=True # Shuffle data since it's ordered chronologically
                                                   )
X_train.head()

,latitude,longitude,category_Abandoned Vehicle,category_Blocked Street or SideWalk,category_Catch Basin Maintenance,category_Color Curb,category_DPW Volunteer Programs,category_Damaged Property,category_Encampments,category_General Request -,category_General Request - 311CUSTOMERSERVICECENTER,category_General Request - AIRPORT SFO,category_General Request - ANIMAL CARE CONTROL,category_General Request - ART COMMISSION,category_General Request - ASSESSOR RECORDER,category_General Request - BOARD OF APPEALS,category_General Request - BOARD OF SUPERVISORS,category_General Request - BUILDING INSPECTION,category_General Request - COUNTY CLERK,category_General Request - DEM,category_General Request - DPH,category_General Request - DPT,category_General Request - DTIS,category_General Request - FIRE DEPARTMENT,category_General Request - HUMAN SERVICES AGENCY,category_General Request - MOH,category_General Request - MONS,category_General Request - MTA,category_General Request - PLANNING,category_General Request - PORT AUTHORITY,category_General Request - PUBLIC WORKS,category_General Request - PUC,category_General Request - RPD,category_General Request - SFPD,category_Graffiti,category_Homeless Concerns,category_Illegal Postings,category_Litter Receptacles,category_MUNI Feedback,category_Noise Report,category_Parking Enforcement,category_Rec and Park Requests,category_Residential Building Request,category_SFHA Requests,category_Sewer Issues,category_Sidewalk or Curb,category_Sign Repair,category_Street Defects,category_Street and Sidewalk Cleaning,category_Streetlights,category_Temporary Sign Request,category_Tree Maintenance,street_10TH ST,street_11TH AVE,street_11TH ST,street_12TH AVE,street_12TH ST,street_13TH ST,street_14TH AVE,street_14TH ST,street_15TH AVE,street_15TH ST,street_16TH AVE,street_16TH ST,street_1750 MCALLISTER ST,street_17TH AVE,street_17TH ST,street_18TH AVE,street_18TH ST,street_19TH AVE,street_19TH ST,street_1ST ST,street_20TH AVE,street_20TH ST,street_21ST AVE,street_21ST ST,street_227 BAY STREET,street_22ND AVE,street_22ND ST,street_23RD AVE,street_23RD ST,street_24TH AVE,street_24TH ST,street_24th and York Mini Park,street_25TH AVE,street_25TH ST,street_26TH AVE,street_26TH ST,street_27TH AVE,street_27TH ST,street_28TH AVE,street_28TH ST,street_29TH AVE,street_29TH ST,street_2ND AVE,street_2ND ST,street_30TH AVE,street_30TH ST,street_31ST AVE,street_320-330 CLEMENTINA,street_32ND AVE,street_33RD AVE,street_34TH AVE,street_35TH AVE,street_36TH AVE,street_37TH AVE,street_38TH AVE,street_39TH AVE,street_3RD AVE,street_3RD ST,street_40TH AVE,street_41ST AVE,street_42ND AVE,street_43RD AVE,street_44TH AVE,street_45TH AVE,street_462 DUBOCE,street_46TH AVE,street_47TH AVE,street_48TH AVE,street_4TH AVE,street_4TH ST,street_5TH AVE,street_5TH ST,street_6TH AVE,street_6TH ST,street_7TH AVE,street_7TH ST,street_8TH AVE,street_8TH ST,street_8TH TI ST,street_9TH AVE,street_9TH ST,street_ABBEY ST,street_ACORN ALY,street_ACTON ST,street_ADA CT,street_ADAIR ST,street_ADDISON ST,street_AGNON AVE,street_AGUA WAY,street_AHERN WAY,street_AL SCOMA WAY,street_ALABAMA ST,street_ALAMEDA ST,street_ALANA WAY,street_ALBERTA ST,street_ALBION ST,street_ALDER ST,street_ALEMANY BLVD,street_ALEMANY BLVD OFF RAMP,street_ALHAMBRA ST,street_ALICE B TOKLAS PL,street_ALLISON ST,street_ALLSTON WAY,street_ALOHA AVE,street_ALPHA ST,street_ALPINE TER,street_ALTA MAR WAY,street_ALTA ST,street_ALTON AVE,street_ALVARADO ST,street_ALVISO ST,street_AMADOR ST,street_AMAZON AVE,street_AMBROSE BIERCE ST,street_AMHERST ST,street_ANDERSON ST,street_ANDOVER ST,street_ANGLO ALY,street_ANKENY ST,street_ANNIE ST,street_ANTHONY ST,street_ANTONIO ST,street_ANZA ST,street_ANZAVISTA AVE,street_APPLETON AVE,street_APTOS AVE,street_AQUAVISTA WAY,street_ARAGO ST,street_ARBALLO DR,street_ARBOR ST,street_ARCH ST,street_ARCO WAY,street_ARELIOUS WALKER DR,street_ARGENT ALY,street_ARGONAUT AVE,street_ARGUELLO BLVD,street_ARKANSAS ST,street_ARLETA AVE,street_ARLINGTON ST,stre

In [17]:
print('df\t', df.shape)
print('X_train\t', X_train.shape)
print('X_test\t', X_test.shape)
print('y_train\t', y_train.shape)
print('y_test\t', y_test.shape)

df	 (16786, 19)
X_train	 (13428, 1640)
X_test	 (3358, 1640)
y_train	 (13428,)
y_test	 (3358,)


## Class Balancing

In [4]:
# Target variable
target_count = df['is_duplicate'].value_counts()

# Print class balance
print(f'Class 0: {target_count[0]}')
print(f'Class 1: {target_count[1]}')
print(f'Proportion: {round(target_count[0] / target_count[1], 2)} : 1')
print(f'Percentage of Majority Class: {round(target_count[0] / sum(target_count), 3)*100}')

Class 0: 15824
Class 1: 962
Proportion: 16.45 : 1
Percentage of Majority Class: 94.3


In [19]:
# A ratio of .5 is saying that 50% of my data is simulated
# Trevor noted that .2 would be good but let's try different ratios
X_smoted, y_smoted = SMOTE(random_state=2020).fit_sample(X_train, y_train)
Counter(y_smoted)
# pd.Series(y_smoted).value_counts().plot.bar()

Counter({0: 12659, 1: 12659})

In [20]:
# Add cross validation

# Modeling

In [21]:
# Istantiate the model
xgb_clf = XGBClassifier()

# Construct X_smoted dataframe
X_smoted_df = pd.DataFrame(X_smoted, columns=X_test.columns)

# Train the model
xgb_clf.fit(X_smoted_df, y_smoted)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [22]:
# Make prediction on training data
train_pred_smote = xgb_clf.predict(X_smoted_df)

# # Make prediction on test data
# test_pred_smote = xgb_clf.predict(X_test)

In [23]:
print('Training Accuracy Score = ', round(accuracy_score(train_pred_smote, y_smoted), 4)*100)
# print('Tests = ', round(accuracy_score(test_pred_smote, y_test), 4)*100)

Training Accuracy Score =  84.11


In [ ]:
# Print confusion matrix for XGBoost
xgb_confusion = confusion_matrix(y_smoted, train_pred_smote)

plt.figure(dpi=125)
sns.heatmap(xgb_confusion, annot=True, fmt='g', square=True, cbar=False,
            xticklabels=['no duplicate', 'is duplicate'],
            yticklabels=['no duplicate', 'is duplicate'])

plt.title('Confusion Matrix - Training Dataset\nXGBoost', pad=20)
plt.xlabel('Predicted\n', labelpad=20)
plt.ylabel('Actual\n', labelpad=20);

# Feature Engineering

In [ ]:
df.head()

* Cosine similarity of service_subtype
* Time of day (hourly, bins)
* Day of the week
* Date
* Block type / building permit
* Lat/long rounded to 3rd decimal place
* Zip code
* Duplication ratio by category or something similar
* 

In [ ]:
# Test rounding the lat/long
df['latitude'] = round(df['latitude'], 4)
df['longitude'] = round(df['longitude'], 4)

## Confusion Matrix for Test Dataset

In [ ]:
# # Print confusion matrix for XGBoost – TEST Dataset
# xgb_confusion = confusion_matrix(y_test, test_pred_smote)

# plt.figure(dpi=125)
# sns.heatmap(xgb_confusion, annot=True, fmt='g', square=True, cbar=False,
#             xticklabels=['no duplicate', 'is duplicate'],
#             yticklabels=['no duplicate', 'is duplicate'])

# plt.title('Confusion Matrix - Test Dataset\nXGBoost', pad=20)
# plt.xlabel('Predicted\n', labelpad=20)
# plt.ylabel('Actual\n', labelpad=20);